## CRUD Example w/ Price Alerts

This short sample demonstrates Create-Read-Update-Delete (CRUD) operations with OpenAPI for price alerts.

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()
client.login()

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID: 2757e3a96e9c4687a498ae9bc86c2c86)


In [2]:
# retrieve current price alerts (in this case there are no alerts configured)
alerts = client.get("/vas/v1/pricealerts/definitions")
print(f"These price alerts are currently configured: {[alert for alert in alerts['Data']]}")

These price alerts are currently configured: []


## Create

In [3]:
# create new price alert: AAPL traded above 200 USD on default account, active for the next 10 days
from datetime import datetime, timedelta

account_id = client.get("/port/v1/clients/me")["DefaultAccountId"]

expiry = datetime.now() + timedelta(days=10)

new_alert = {
    "AccountId": account_id,
    "Uic": 211,
    "AssetType": "Stock",
    "IsRecurring": False,
    "Operator": "GreaterOrEqual",
    "PriceVariable": "Traded",
    "TargetValue": "200",
    "ExpiryDate": str(expiry),
}

created_alert = client.post("/vas/v1/pricealerts/definitions", data=new_alert)
alert_id = created_alert["AlertDefinitionId"]
print(f"New price alert was configured with ID: {alert_id}")

New price alert was configured with ID: 883953


## Read

In [4]:
# retrieve the collection of price alerts to confirm the new alert has been created

client.get("/vas/v1/pricealerts/definitions")

{'__count': 1,
 'Data': [{'AccountId': '16371609_USD',
   'AlertDefinitionId': '883953',
   'AssetType': 'Stock',
   'ClientId': '16371609',
   'Comment': '',
   'ExpiryDate': '2023-06-28T09:52:48.090000Z',
   'IsExtendedHours': False,
   'IsRecurring': False,
   'Operator': 'GreaterOrEqual',
   'PriceVariable': 'Traded',
   'State': 'Enabled',
   'TargetValue': 200.0,
   'Uic': 211,
   'UserId': '16371609'}]}

## Update

In [5]:
# update price alert (price is now 210 USD)
updated_alert = {
    "AccountId": account_id,
    "AssetType": "Stock",
    "IsRecurring": False,
    "Operator": "GreaterOrEqual",
    "PriceVariable": "Traded",
    "TargetValue": "210",
    "Uic": 211,
    "ExpiryDate": "2022-12-31T00:00:00Z",
}

client.put(f"/vas/v1/pricealerts/definitions/{alert_id}", data=updated_alert)

# no response here because PUT status code is 204

In [6]:
# retrieve updated alert, price is indeed updated to 210
alert = client.get(f"/vas/v1/pricealerts/definitions/{alert_id}")
print(f"Target value of price alert {alert_id} is now: {alert['TargetValue']}")

Target value of price alert 883953 is now: 210.0


## Delete

In [7]:
# remove alert
client.delete(f"/vas/v1/pricealerts/definitions/{alert_id}")

# again no response because status code is always 204

In [8]:
# alert is indeed removed
client.get("/vas/v1/pricealerts/definitions/")
print(f"These price alerts are currently configured: {[alert for alert in alerts['Data']]}")

These price alerts are currently configured: []
